In [12]:
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

import os

In [3]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [4]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
    <link name="table_top_link">
      <inertial>
        <mass>18.70</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

In [5]:
table_top_sdf_file

'/tmp/robotlocomotion_drake_K3Gayy/table_top.sdf'

# Adding Models 

In this exercise, you will learn the basics of adding objects into Drake. You will learn about URDFs, SDFs, and obj files. 

Parts of this notebook were taken from the [Authoring a Multibody Simulation tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-add293478aac40a984845aa3705eefdd?). Still, it is useful to take a look at the tutorial to learn more about the concepts.

A high level overview of the different formats around describing simulations in Drake:
- URDF and SDF are XML formats that can specify the kinematic and dynamic properties of robots or objects. SDF is the more scalable format, and it is the format that is recommended. However, they should be mostly interchangeable with small syntax differences.
- .obj on the other hand, is a file format that describes the *geometry* of an object, such as a mesh of the surface of the object.
  - There exist other geometry formats, such as .stl, and .dae. These are not well-supported by Drake directly, but [Meshlab](https://www.meshlab.net/), an open-source software, is a handy tool to convert common formats to a `.obj` for you.
- [Model directives](https://drake.mit.edu/doxygen_cxx/structdrake_1_1multibody_1_1parsing_1_1_model_directive.html) are a YAML specification that makes it easy to load multiple URDF and SDF objects into a simulation. They're used at the bottom of this notebook.


The most important formats for creating multibody scenarios in Drake are the [Unified Robot Description Format (URDF)](http://wiki.ros.org/urdf) and the [Simulation Description Format (SDFormat)](http://sdformat.org/).

They are both XML formats to describe robots or objects for robot simulators or visualization, and are fairly similar in syntax.

In a high-level sense, you express different components of your robot using `<link>` tags and connect them via `<joint>` tags. Each `<link>` has three major subtags, `<visual>`, `<collision>`, and `<inertial>`, for its visualization, planning/collision checking, and dynamics aspects. For `<visual>` and `<collision>`, you can either use primitives (box, sphere, cylinder, etc.) or meshes (.obj) to represent the underlying geometry.

The supported primitive types and the supported file types for meshes are described [here](https://drake.mit.edu/doxygen_cxx/group__geometry__file__formats.html). 

### Written Question
Compare and contrast two SDF files of the same YCB sugar box: [File 1](https://github.com/RobotLocomotion/drake/blob/8ed3a0264c521ffc1ea8861f5701b81dfb696af8/manipulation/models/ycb/sdf/004_sugar_box.sdf) and [File 2](https://github.com/RussTedrake/manipulation/blob/24f122591f72cc78931ea3f461d0c34845dc9aca/manipulation/models/hydro/004_sugar_box.sdf). File 2 uses hydroelastic contact, and file 1 uses the default point contact model.

[Hydroelastic contact](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html) in Drake can be used to create compliant collision geometries. This is a great tool for improving the contact dynamics between simulated objects, compared to traditional rigid contact geometries.

Answer the following questions in your written submission.

**Part (a):**  How is the collision geometry (not contact models!) different in File 1 than File 2? What tags enable hydroelastic contact in File 2?

**Part (b):**  In both files, we use the sugar box obj mesh as the visual geometry. For File 2, we could have used the obj mesh as the collision geometry too. In one sentence, why might we prefer to specify the collision geometry using primitives, rather than using the obj mesh? 

## Adding your own models

Now, you will add an object of your choosing into simulation! You have a lot of freedom here. Ideally, this object can be used in your final project. It doesn't have to be an object your robot can pick up; it can be anything you want in your simulation. 

- The only requirement is it cannot be directly taken from the existing models in Drake without modification.
- Your model file can be in any of the formats supported by the [Parser](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_parser.html), but we recommend using SDF. Your model should use hydroelastic contact.

One option is to find a 3D model online, such as at [TurboSquid](https://www.turbosquid.com/3d-model/free), and convert it into a .obj file. The .obj file can then be directly turned into a SDF file by using the below terminal command, documented [here](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html). On Deepnote, you can open a terminal on the left bar and run the command after you have uploaded your .obj file. 
```sh
python3 -m pydrake.multibody.mesh_to_model path/to/mesh.obj
```
We have a more advanced version of this tool available in `manipulation/create_sdf_from_mesh.py`, which additionally supports convex decomposition of a nonconvex mesh and mesh simplification. This isn't needed for this problem, but may be very useful for your project.

Another option is to purely use SDF and geometric primitives to compose primitives (and maybe meshes) to build up your object. 

Here's some more examples of various SDF files to use as inspiration or a guide: 
- [Rubik's Cube](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/02-Lets-get-you-a-robot-8f86172b-b597-4ceb-9bad-92d11ac7a6cc/notebook/rubiks_cube-35164353b13d4a47910ca14e588c74d6?)
- [Drake examples](https://github.com/RobotLocomotion/drake/tree/master/examples), which includes things such as a bouncing ball. 
- If there was a [final project](https://manipulation.csail.mit.edu/misc.html#projects) that had objects you'd like to use, see if their code is open source or ask!

Once you have created your model file, fill in the **absolute path** below. On Deepnote, there is an option on the left bar to copy paths into your clipboard. 

**Part (c):** In your written submission, briefly describe what your object is. Take a screenshot of your object on the table in simulation (the final code cell). Your screenshot should include red contact force arrows and not green contact force arrows, which indicate that you are using hydroelastic contact. 

In [6]:
cwd = os.getcwd()
your_model_filename = cwd+"/Board.sdf" # Write the absolute path to your file here


It is very helpful to visualize and check your model prior to putting it into simulation. Run the below cell to do so. 

Under the `Scene > drake` dropdown in MeshCat, you can enable viewing 'illustration', 'inertia', and 'proximity' to see the visual model, inertia matrix, and collision model, respectively. You can also adjust the poses of the links in the sliders. 

In [ ]:
if your_model_filename:
    visualizer = ModelVisualizer(meshcat=meshcat)
    visualizer.parser().AddModels(your_model_filename)
    visualizer.Run(loop_once=not running_as_notebook)

INFO:drake:Click 'Stop Running' or press Esc to quit


In [7]:
def convert_to_coords(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return "" + str(.125*(y-3.5)) + ", "+ str(.125*(x-3.5)) 

def add_piece(piece_id, name,  pos):
    return f"""
- add_model:
    name: {piece_id}
    file: file://{cwd}/{name}.sdf
    default_free_body_pose:
        {name}:
            translation: [{convert_to_coords(pos)}, 5.5]
            rotation: !Rpy {{ deg: [0, 0, 0] }}"""


Now, we'll create the simulation with your model. The simulation is replayable in MeshCat. A YCB sugar box has been added to the simulation as a reference, but you may comment it out and remove it.

You will have to replace the link specified in the model directives based on your file. You may have to adjust the starting pose of your model, physical properties of your model, and/or the size of the table (defined at the top of the file) in order to not have your object fall through the world.

In [11]:
def create_scene():
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://{table_top_sdf_file}
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
    if your_model_filename:
        scenario_data += f"""
- add_model:
    name: articulated_worm
    file: file://{your_model_filename}
    default_free_body_pose:
        Board:
            translation: [0, 0, 2.5]
            rotation: !Rpy {{ deg: [90, 0, 0] }}    
"""
    scenario_data += add_piece('p1', 'BlackKnight',  'b8')
    scenario_data += add_piece('p2', 'BlackBishop',  'c8')
    scenario_data += add_piece('p3', 'BlackKnight',  'g8')
    scenario_data += add_piece('p4', 'BlackBishop',  'f8')
    scenario_data += add_piece('p5', 'BlackQueen',  'd8')
    scenario_data += add_piece('p6', 'BlackRook',  'a8')
    scenario_data += add_piece('p7', 'BlackRook',  'h8')
    scenario_data += add_piece('p8', 'BlackKing',  'e8') 
    scenario_data += add_piece('p9', 'WhiteKnight',  'b1')
    scenario_data += add_piece('p10', 'WhiteBishop',  'c1')
    scenario_data += add_piece('p11', 'WhiteKnight',  'g1')
    scenario_data += add_piece('p12', 'WhiteBishop',  'f1')
    scenario_data += add_piece('p13', 'WhiteQueen',  'd1')
    scenario_data += add_piece('p14', 'WhiteRook',  'a1')
    scenario_data += add_piece('p15', 'WhiteRook',  'h1')
    scenario_data += add_piece('p16', 'WhiteKing',  'e1')
    for i in range(8): 
       scenario_data += add_piece(f"p{17+i}", 'WhitePawn',  f"{chr(97+i)}2")
       scenario_data += add_piece(f"p{25+i}", 'BlackPawn',  f"{chr(97+i)}7")

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


create_scene()

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [13]:
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import running_as_notebook

In [14]:
def setup_manipulation_station():
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="package://manipulation/manipulation_station_with_cupboard.dmd.yaml",
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(
        wsg_position.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)

    return initial_pose


# Get initial pose of the gripper by using default context of manip station.
initial_pose = setup_manipulation_station()

p_WR = np.array([0.7477, -0.1445, 0.4148])  # frame R: center of left door.

p_Rhandle = np.array(
    [-0.033, 0.1245, 0]
)  # handle: frame attached to right handle.
p_Whandle = p_WR + p_Rhandle

p_Rhinge = np.array(
    [0.008, -0.1395, 0]
)  # hinge: frame attached to right hinge.
p_Whinge = p_WR + p_Rhinge

p_Rhinge_handle = p_Rhandle - p_Rhinge
r_Rhinge_handle = np.linalg.norm(
    p_Rhandle - p_Rhinge
)  # distance between handle and hinge.

theta_Rhinge_handle = np.arctan2(p_Rhinge_handle[1], p_Rhinge_handle[0])
angle_end = (
    np.pi
)  # end of angle. Decrease to 120~160 deg for the easy version.


# Interpolate pose for opening doors.
def InterpolatePoseOpen(t):
    # Start by interpolating the yaw angle of the hinge.
    angle_start = theta_Rhinge_handle
    theta = angle_start + (angle_end - angle_start) * t
    # Convert to position and rotation.
    p_Whandle = (
        r_Rhinge_handle * np.array([np.cos(theta), np.sin(theta), 0])
        + p_Whinge
    )
    # Add some offset here to account for gripper yaw angle.
    R_Whandle = RollPitchYaw(0, 0, theta).ToRotationMatrix()
    X_Whandle = RigidTransform(R_Whandle, p_Whandle)

    # Add a little offset to account for gripper.
    p_handleG = np.array([0.0, 0.1, 0.0])
    R_handleG = RollPitchYaw(0, np.pi, np.pi).ToRotationMatrix()
    X_handleG = RigidTransform(R_handleG, p_handleG)
    X_WG = X_Whandle.multiply(X_handleG)
    return X_WG


## Interpolate Pose for entry.
def make_gripper_orientation_trajectory():
    traj = PiecewiseQuaternionSlerp()
    traj.Append(0.0, initial_pose.rotation())
    traj.Append(5.0, InterpolatePoseOpen(0.0).rotation())
    return traj


def make_gripper_position_trajectory():
    traj = PiecewisePolynomial.FirstOrderHold(
        [0.0, 5.0],
        np.vstack(
            [
                [initial_pose.translation()],
                [InterpolatePoseOpen(0.0).translation()],
            ]
        ).T,
    )
    return traj


entry_traj_rotation = make_gripper_orientation_trajectory()
entry_traj_translation = make_gripper_position_trajectory()


def InterpolatePoseEntry(t):
    return RigidTransform(
        RotationMatrix(entry_traj_rotation.orientation(t)),
        entry_traj_translation.value(t),
    )


# Wrapper function for end-effector pose. Total time: 11 seconds.
def InterpolatePose(t):
    if t < 5.0:
        # Duration of entry motion is set to 5 seconds.
        return InterpolatePoseEntry(t)
    elif (t >= 5.0) and (t < 6.0):
        # Wait for a second to grip the handle.
        return InterpolatePoseEntry(5.0)
    else:
        # Duration of the open motion is set to 5 seconds.
        return InterpolatePoseOpen((t - 6.0) / 5.0)


# Visualize our end-effector nominal trajectory.
t_lst = np.linspace(0, 11, 30)
pose_lst = []
for t in t_lst:
    AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=0.2)
    pose_lst.append(InterpolatePose(t))

# Create gripper trajectory.
gripper_t_lst = np.array([0.0, 5.0, 6.0, 11.0])
gripper_knots = np.array([0.05, 0.05, 0.0, 0.0]).reshape(1, 4)
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_t_lst, gripper_knots)


def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and gripper, used for controllers."""
    robot_sdf_path = "package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf"
    gripper_sdf_path = "package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelsFromUrl(robot_sdf_path)
    parser.AddModelsFromUrl(gripper_sdf_path)
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.world_frame(),
        frame_on_child_M=plant_robot.GetFrameByName("iiwa_link_0"),
    )
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.GetFrameByName("iiwa_link_7"),
        frame_on_child_M=plant_robot.GetFrameByName("body"),
        X_FM=RigidTransform(
            RollPitchYaw(np.pi / 2, 0, np.pi / 2), np.array([0, 0, 0.114])
        ),
    )
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant_robot.Finalize()

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index()
        )

    return plant_robot, link_frame_indices


def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="package://manipulation/manipulation_station_with_cupboard.dmd.yaml",
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
    )
    builder.Connect(
        g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    simulator = Simulator(diagram)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(duration)
    visualizer.PublishRecording()

    return simulator, plant

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/fe5326c5ffc36fda12c58883d22d29dc86009d65.tar.gz


In [ ]:
def create_q_knots(pose_lst):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant, _ = CreateIiwaControllerPlant()
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")
    q_nominal = np.array(
        [0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0]
    )  # nominal joint angles for joint-centering.

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )

    for i in range(len(pose_lst)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        #### Modify here ###############################
        # Lol turns out we don't actually need a constraint here
        AddPositionConstraint(ik, pose_lst[i].translation(), pose_lst[i].translation())
        # add rotation constraint
        AddOrientationConstraint(ik, pose_lst[i].rotation(), 3.14159265/30)
        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
        else:
            prog.SetInitialGuess(q_variables, q_knots[i-1])

        ################################################

        result = Solve(prog)

        assert result.is_success()

        q_knots.append(result.GetSolution(q_variables))

    return q_knots

In [ ]:
q_knots = np.array(create_q_knots(pose_lst))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)
simulator, station_plant = BuildAndSimulateTrajectory(q_traj, g_traj, 11.0)

## How will this notebook be Graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

This notebook only consists of written questions, so please submit all responses in your PDF submission to Gradescope. For this exercise, you should have completed the following subparts:
- Correct answers to part (a)
- Correct answers to part (b)
- A screenshot and description of your added object for part (c)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8432d35-2370-4de9-924c-af3802cf315d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>